In [1]:
import numpy as np
import matplotlib.pyplot as plt

# You may change the mhealth_activity module but your algorithm must support the original version
from mhealth_activity import Recording, Trace, Activity, WatchLocation, Path

# For interactive plots, uncomment the following line
# %matplotlib widget
import os
import pandas as pd
import pickle
from tqdm import tqdm
from scipy.fft import fft, fftfreq
import scipy.stats as stats
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error,accuracy_score,precision_score,recall_score,confusion_matrix,classification_report,f1_score


In [2]:
#read data

data = pd.read_pickle('data/pickled_and_sorted_training_data.pkl.zst')


In [76]:
# for x in data['altitude']:
#     print(x.values)

test = [x.values for x in data['altitude']]
print(test)
print(test[0])

[array([401.06787109, 401.06787109, 401.06787109, ..., 450.23861323,
       450.23861314, 450.23861314]), array([450.58095942, 450.58095942, 450.58095942, ..., 407.91178428,
       407.91178428, 407.91178428]), array([408.7502594 , 408.7502594 , 407.29495239, ..., 450.72974694,
       450.72974694, 450.72974694]), array([459.9460144 , 459.9460144 , 459.9460144 , ..., 406.84563044,
       406.84563044, 406.84563044]), array([461.61029053, 461.61029053, 461.61029053, ..., 496.6081543 ,
       496.6081543 , 496.6081543 ]), array([403.46264648, 403.46264648, 403.46264648, ..., 451.20713487,
       451.20713487, 451.20713487]), array([452.00967789, 452.00967789, 452.0096817 , ..., 407.40448512,
       407.40448512, 407.40448512]), array([454.3999939 , 454.3999939 , 454.3999939 , ..., 507.79998779,
       507.79998779, 507.79998779]), array([456.29998779, 456.29998779, 456.29998779, ..., 502.40064579,
       502.40064579, 502.40064579]), array([453.10933685, 453.10933685, 453.10933685, ..., 

In [77]:
# def features_extraction_watchpos(df, prefix): 
    
#     FEATURES = ['MIN','MAX','MEAN','RMS','VAR','STD','POWER','PEAK','P2P','CREST FACTOR','SKEW','KURTOSIS',
#             'MAX_f','SUM_f','MEAN_f','VAR_f','PEAK_f','SKEW_f','KURTOSIS_f']

#     for idx, feature in enumerate(FEATURES):
#         FEATURES[idx] = FEATURES[idx] + '_' + prefix
    
#     print(FEATURES)
#     Min=[];Max=[];Mean=[];Rms=[];Var=[];Std=[];Power=[];Peak=[];Skew=[];Kurtosis=[];P2p=[];CrestFactor=[];
#     FormFactor=[]; PulseIndicator=[];
#     Max_f=[];Sum_f=[];Mean_f=[];Var_f=[];Peak_f=[];Skew_f=[];Kurtosis_f=[]
    
#     X = df.values
#     ## TIME DOMAIN ##
#     #list of lists of lists, ugly as fuck but it works 
#     for recording in X:
#         Min.append(np.min(recording[0]))
#         Max.append(np.max(recording[0]))
#         Mean.append(np.mean(recording[0]))
#         Rms.append(np.sqrt(np.mean(recording[0]**2)))
#         Var.append(np.var(recording[0]))
#         Std.append(np.std(recording[0]))
#         Power.append(np.mean(recording[0]**2))
#         Peak.append(np.max(np.abs(recording[0])))
#         P2p.append(np.ptp(recording[0]))
#         CrestFactor.append(np.max(np.abs(recording[0]))/np.sqrt(np.mean(recording[0]**2)))
#         Skew.append(stats.skew(recording[0]))
#         Kurtosis.append(stats.kurtosis(recording[0]))
#         FormFactor.append(np.sqrt(np.mean(recording[0]**2))/np.mean(recording[0]))
#         PulseIndicator.append(np.max(np.abs(recording[0]))/np.mean(recording[0]))
#         ## FREQ DOMAIN ##
#         ft = fft(recording[0])
#         S = np.abs(ft**2)/len(df)
#         Max_f.append(np.max(S))
#         Sum_f.append(np.sum(S))
#         Mean_f.append(np.mean(S))
#         Var_f.append(np.var(S))
        
#         Peak_f.append(np.max(np.abs(S)))
#         Skew_f.append(stats.skew(recording[0]))
#         Kurtosis_f.append(stats.kurtosis(recording[0]))

    
#     #Create dataframe from features
#     df_features = pd.DataFrame(index = [FEATURES], 
#                                data = [Min,Max,Mean,Rms,Var,Std,Power,Peak,P2p,CrestFactor,Skew,Kurtosis,
#                                        Max_f,Sum_f,Mean_f,Var_f,Peak_f,Skew_f,Kurtosis_f])
#     return df_features

def features_extraction_common(df, prefix): 
    
    FEATURES = ['Mean', 'STD', 'MAD', 'Var', 'Min', 'Max', 'SMA', 'Energy', 'IQR', 'Entropy', 'Sum_f', 'Max_f', 'Peak_f', 'Peak_i_f', 'Mean_f', 'Skew_f', 'Kurtosis_f', 'Mean_d', 'STD_d', 'MAD_d', 'Var_d', 'Min_d', 'Max_d', 'SMA_d', 'Energy_d', 'IQR_d', 'Entropy_d']

    for idx, feature in enumerate(FEATURES):
        FEATURES[idx] = FEATURES[idx] + '_' + prefix
    
    print(FEATURES)
    Min=[];Max=[];Mean=[];Mad=[];Sma=[];Eng=[];Iqr=[];Entr=[];Std=[];Var=[];
    Min_d=[];Max_d=[];Mean_d=[];Mad_d=[];Sma_d=[];Eng_d=[];Iqr_d=[];Entr_d=[];Std_d=[];Var_d=[];
    Max_f=[];Peak_f=[];Peak_i_f=[];Mean_f=[];Skew_f=[];Kurtosis_f=[];Sum_f=[];
    
    X = df.values
    ## TIME DOMAIN ##
    #list of lists of lists, ugly as fuck but it works 
    for recording in X:
        Mean.append(np.mean(recording[0]))
        Std.append(np.std(recording[0]))
        #median absolute deviation
        Mad.append(stats.median_abs_deviation(recording[0], scale=1))
        Var.append(np.var(recording[0]))
        Min.append(np.min(recording[0]))
        Max.append(np.max(recording[0]))
        #Signal Magnitude Area
        Sma.append(np.sum(recording[0]))
        #energy measure
        Eng.append(np.sum(recording[0]**2)/len(recording[0]))
        Iqr.append(stats.iqr(recording[0]))
        Entr.append(stats.entropy(recording[0]))

        ## FREQ DOMAIN ##
        ft = np.abs(fft(recording[0]))
        Sum_f.append(np.sum(ft))
        Max_f.append(np.max(ft))
        Peak_f.append(np.max((ft)))
        Peak_i_f.append(np.argmax(ft))
        Mean_f.append(np.mean(ft))
        Skew_f.append(stats.skew(ft))
        Kurtosis_f.append(stats.kurtosis(ft))

        #derivative
        f = np.gradient(recording[0])
        Mean_d.append(np.mean(recording[0]))
        Std_d.append(np.std(recording[0]))
        #median absolute deviation
        Mad_d.append(stats.median_abs_deviation(recording[0], scale=1))
        Var_d.append(np.var(recording[0]))
        Min_d.append(np.min(recording[0]))
        Max_d.append(np.max(recording[0]))
        #Signal Magnitude Area
        Sma_d.append(np.sum(recording[0]))
        #energy measure
        Eng_d.append(np.sum(recording[0]**2)/len(recording[0]))
        Iqr_d.append(stats.iqr(recording[0]))
        Entr_d.append(stats.entropy(recording[0]))

    #Create dataframe from features
    df_features = pd.DataFrame(index = [FEATURES], 
                               data = [Mean,Std, Mad, Var, Min, Max, Sma, Eng, Iqr, Entr, Sum_f, Max_f, Peak_f, Peak_i_f, Mean_f, Skew_f, Kurtosis_f, Mean_d ,Std_d, Mad_d, Var_d, Min_d, Max_d, Sma_d, Eng_d, Iqr_d, Entr_d])
    return df_features

# def features_extraction_path(df, prefix): 



In [48]:
#load pickled training 3d norm accelerometer data
file = open('data/accel_mag_train.pkl', 'rb')
pickled = pickle.load(file)
accel_mag_train =  pd.DataFrame(((x,) for x in pickled), columns=['lists'])
print(accel_mag_train.shape)
file.close()

file = open('data/magneto_mag_train.pkl', 'rb')
pickled = pickle.load(file)
magneto_mag_train =  pd.DataFrame(((x,) for x in pickled), columns=['lists'])
print(magneto_mag_train.shape)
file.close()

file = open('data/gyro_mag_train.pkl', 'rb')
pickled = pickle.load(file)
gyro_mag_train =  pd.DataFrame(((x,) for x in pickled), columns=['lists'])
print(gyro_mag_train.shape)
file.close()

pos_labels  = []
path_labels = []
for label in data["labels"]:
    path_labels.extend([label["path_idx"]])
    pos_labels.extend([label["watch_loc"]])


temp_train =  pd.DataFrame(((x.values,) for x in data['temperature']), columns=['temp'])

(396, 1)
(396, 1)
(396, 1)


In [61]:
#extract features from accelerometer norm and get rid of multiindex
features_acc = pd.DataFrame.transpose(features_extraction_common(accel_mag_train, "acc"))
features_acc.columns = features_acc.columns.map(''.join)

features_mag = pd.DataFrame.transpose(features_extraction_common(magneto_mag_train, "mag"))
features_mag.columns = features_mag.columns.map(''.join)

features_gyro = pd.DataFrame.transpose(features_extraction_common(gyro_mag_train, "gyro"))
features_gyro.columns = features_gyro.columns.map(''.join)

features_temp = pd.DataFrame.transpose(features_extraction_common(temp_train, "temp"))
features_temp.columns = features_temp.columns.map(''.join)



['Mean_acc', 'STD_acc', 'MAD_acc', 'Var_acc', 'Min_acc', 'Max_acc', 'SMA_acc', 'Energy_acc', 'IQR_acc', 'Entropy_acc', 'Sum_f_acc', 'Max_f_acc', 'Peak_f_acc', 'Peak_i_f_acc', 'Mean_f_acc', 'Skew_f_acc', 'Kurtosis_f_acc']
['Mean_mag', 'STD_mag', 'MAD_mag', 'Var_mag', 'Min_mag', 'Max_mag', 'SMA_mag', 'Energy_mag', 'IQR_mag', 'Entropy_mag', 'Sum_f_mag', 'Max_f_mag', 'Peak_f_mag', 'Peak_i_f_mag', 'Mean_f_mag', 'Skew_f_mag', 'Kurtosis_f_mag']
['Mean_gyro', 'STD_gyro', 'MAD_gyro', 'Var_gyro', 'Min_gyro', 'Max_gyro', 'SMA_gyro', 'Energy_gyro', 'IQR_gyro', 'Entropy_gyro', 'Sum_f_gyro', 'Max_f_gyro', 'Peak_f_gyro', 'Peak_i_f_gyro', 'Mean_f_gyro', 'Skew_f_gyro', 'Kurtosis_f_gyro']
['Mean_temp', 'STD_temp', 'MAD_temp', 'Var_temp', 'Min_temp', 'Max_temp', 'SMA_temp', 'Energy_temp', 'IQR_temp', 'Entropy_temp', 'Sum_f_temp', 'Max_f_temp', 'Peak_f_temp', 'Peak_i_f_temp', 'Mean_f_temp', 'Skew_f_temp', 'Kurtosis_f_temp']


In [66]:
#acc + mag yield the best results(87%), acc + mag + gyro 85%, mag + gyro 85%, acc + gyro  82.5%, 
features = pd.concat([features_acc, features_mag, features_gyro, features_temp], axis=1)
features

,Mean_acc,STD_acc,MAD_acc,Var_acc,Min_acc,Max_acc,SMA_acc,Energy_acc,IQR_acc,Entropy_acc,...,Energy_temp,IQR_temp,Entropy_temp,Sum_f_temp,Max_f_temp,Peak_f_temp,Peak_i_f_temp,Mean_f_temp,Skew_f_temp,Kurtosis_f_temp
0,1.202418,0.212961,0.142390,0.045352,0.153406,2.597480,147175.953766,1.491161,0.285437,11.699222,...,1254.348235,1.0,8.942221,355043.852911,270874.0,270874.0,0.0,46.410961,87.386861,7637.927548
1,1.092686,0.357553,0.279282,0.127844,0.046882,2.826338,128552.271322,1.321806,0.563193,11.621486,...,1508.076295,1.0,8.902472,389264.857150,285433.0,285433.0,0.0,52.939597,85.633269,7336.623385
2,1.093724,0.306151,0.161359,0.093728,0.083755,3.057903,128586.928630,1.289960,0.335081,11.638646,...,1320.389630,1.0,8.902099,321953.672448,266983.0,266983.0,0.0,43.815143,85.682121,7340.609131
3,1.026057,0.349516,0.285567,0.122162,0.276498,2.088655,110551.514205,1.174955,0.572351,11.529345,...,1098.726017,1.0,8.814485,321946.224834,223115.0,223115.0,0.0,47.809062,81.939867,6717.677634
4,1.078819,0.297570,0.135679,0.088548,0.080601,3.464102,82870.587453,1.252399,0.274119,11.214906,...,1296.828786,0.0,8.476439,212919.614117,172867.0,172867.0,0.0,44.349014,69.239693,4793.414258
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
391,1.551828,0.640051,0.320108,0.409665,0.093611,3.464102,180334.844977,2.817836,1.059469,11.582066,...,1442.989398,4.0,8.887708,502114.904019,275132.0,275132.0,0.0,69.133265,84.523408,7179.111068
392,1.468600,0.697000,0.259207,0.485809,0.025811,3.464102,166856.432829,2.642595,0.996380,11.534595,...,1257.255175,1.0,8.867663,354837.121528,251703.0,251703.0,0.0,49.970021,84.169891,7087.000106
393,1.294628,0.596891,0.212268,0.356279,0.034486,3.464102,83519.058875,2.032342,0.552558,10.976953,...,1095.879464,0.0,8.301976,177441.151454,133470.0,133470.0,0.0,44.008222,63.468966,4026.539415
394,0.996864,0.410565,0.255059,0.168563,0.032321,3.464066,133165.093887,1.162301,0.507380,11.719029,...,1289.699844,3.0,9.027738,542910.014684,299204.0,299204.0,0.0,65.026951,90.802308,8275.538717


In [64]:
#split the training data because test data doesnt have labels
#why help the students even slightly?

def split_and_train():
    pos_train_features, pos_test_features, pos_train_labels, pos_test_labels = train_test_split(
        features, pos_labels, test_size=0.1, random_state=10)

    scaler = StandardScaler()
    X_train_pos = scaler.fit_transform(pos_train_features) 

    # Instantiate model with 1000 decision trees, use all cores
    rf_pos = RandomForestRegressor(n_estimators = 1000, random_state = 42, n_jobs=-1)
    # Train the model on training data
    rf_pos.fit(X_train_pos, pos_train_labels)

split_and_train()

In [65]:
def evaluate_pos(model, test_features, test_labels):
    
    y_pred = model.predict(test_features)

    #cast to int because indices cant be floats
    watch_pos_pred = np.rint(y_pred)

    num_tests = len(watch_pos_pred)

    watch_pos_mismatches = np.count_nonzero(watch_pos_pred != test_labels)

    print(f"{num_tests} tests, {watch_pos_mismatches} watch pos mismatches {(1-(watch_pos_mismatches/num_tests))*100} accuracy")
    return (1-(watch_pos_mismatches/num_tests))*100


X_val = scaler.fit_transform(pos_test_features) 


evaluate_pos(rf_pos,X_val,pos_test_labels)

40 tests, 4 watch pos mismatches 90.0 accuracy
